# Statistics and exports
In this notebook, we use the results generated by the ```functionalConnectivityRaw.jl``` in jld2 files to calculate a bunch of statistics per pair and export them to a few formats, including ```.js``` files that are underlying the website. This notebook follows the ```code/statsAndExports.jl``` script (exports are commented out as they would be useless in the Binder).

### Preparing the environment
We want to be in ```CX-Functional-Analysis```

In [1]:
cd("..")

### Load packages

In [2]:
using JLD2,DataFrames,AxisArrays,FileIO,CSV
using Interpolations
using StatsBase,Distributions
using JSON
using DataStructures
using Distances,Bootstrap
using HTTP

INFO: Recompiling stale cache file /localhome/romain/.julia/lib/v0.6/HTTP.ji for module HTTP.


In [3]:
include("code/functions/fluoRunUtilities.jl")

category_stats (generic function with 1 method)

### Load data
We're loading the labbook (as a data frame), the table containing the drivers info and the dataset of the fluorescent traces. The jld2 files are on the OSF framework storage, so we need to interact with their API to get the download links.

In [4]:
reqLabbook = HTTP.get("https://api.osf.io/v2/nodes/vsa3z/files/osfstorage/?filter[name]=labbookTable.jld2")
labbookLink = JSON.parse("$(Char.(reqLabbook.body)...)")

reqData = HTTP.get("https://api.osf.io/v2/nodes/vsa3z/files/osfstorage/?filter[name]=rawData.jld2")
rawLink = JSON.parse("$(Char.(reqData.body)...)")

Dict{String,Any} with 3 entries:
  "meta"  => Dict{String,Any}(Pair{String,Any}("version", "2.0"))
  "links" => Dict{String,Any}(Pair{String,Any}("prev", nothing),Pair{String,Any…
  "data"  => Any[Dict{String,Any}(Pair{String,Any}("links", Dict{String,Any}(Pa…

In [5]:
labbook = load(download(labbookLink["data"][1]["links"]["download"]),"labbook")
linesToType = CSV.read("LinesAndTypes.csv")
full_data_dict = load(download(rawLink["data"][1]["links"]["download"]))

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 7766k  100 7766k    0     0  5133k      0  0:00:01  0:00:01 --:--:-- 31.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 27.7M  100 27.7M    0     0  12.3M      0  0:00:02  0:00:02 --:--:-- 79.6M


Dict{String,Any} with 476 entries:
  "jun1417-Fly1-EB"    => Tuple{AxisArrays.AxisArray{Float64,3,Array{Float64,3}…
  "jun1416-Fly2-Gall"  => Tuple{AxisArrays.AxisArray{Float64,3,Array{Float64,3}…
  "jun3017-Fly3-EB"    => Tuple{AxisArrays.AxisArray{Float64,3,Array{Float64,3}…
  "dec0315-Fly2-EB"    => Tuple{AxisArrays.AxisArray{Float64,3,Array{Float64,3}…
  "dec2115-Fly2-EB"    => Tuple{AxisArrays.AxisArray{Float64,3,Array{Float64,3}…
  "jul1616-Fly1-FB"    => Tuple{AxisArrays.AxisArray{Float64,3,Array{Float64,3}…
  "jun1417-Fly3-LAL"   => Tuple{AxisArrays.AxisArray{Float64,3,Array{Float64,3}…
  "dec0215-Fly1-NO"    => Tuple{AxisArrays.AxisArray{Float64,3,Array{Float64,3}…
  "jan1616-Fly2-PB-in" => Tuple{AxisArrays.AxisArray{Float64,3,Array{Float64,3}…
  "jul1016-Fly2-FB"    => Tuple{AxisArrays.AxisArray{Float64,3,Array{Float64,3}…
  "jun2316-Fly9-EB"    => Tuple{AxisArrays.AxisArray{Float64,3,Array{Float64,3}…
  "may1216-Fly3-FB"    => Tuple{AxisArrays.AxisArray{Float64,3,Array{Float

```full_data_dict``` is a dictionary with one entry per experiment (with keys following the naming convention "jun1417-Fly1-EB", the last part being the region imaged). Each entry is an array with each element corresponding to one experimental run. Each element is a tuple of :
- one AxisArray (the fluorescent traces)
- one dictionary of metadata (the stimulus conditions and the time to drug exposure if relevant)

The AxisArray : 

In [6]:
full_data_dict["jun1417-Fly1-EB"][6][1]

3-dimensional AxisArray{Float64,3,...} with axes:
    :time, 0.0:0.101389541031873:16.120937024067807
    :ROI, 1:1
    :page, Base.OneTo(4)
And data, a 160×1×4 Array{Float64,3}:
[:, :, 1] =
 -0.325959 
 -0.0492237
  0.5837   
  1.09956  
  1.278    
  1.44899  
  1.49861  
  1.47023  
  1.39796  
  1.55742  
  1.50124  
  1.42591  
  1.4936   
  ⋮        
  0.654155 
  0.56826  
  0.591088 
  0.581698 
  0.528128 
  0.577004 
  0.540351 
  0.560025 
  0.469868 
  0.510503 
  0.539799 
  0.573391 

[:, :, 2] =
 0.64606 
 0.674336
 0.648349
 0.719679
 0.811932
 0.783593
 0.807181
 0.832807
 0.779946
 0.805026
 0.780405
 0.78335 
 0.74797 
 ⋮       
 0.567291
 0.57697 
 0.56552 
 0.531834
 0.598471
 0.568311
 0.516127
 0.544202
 0.514786
 0.480528
 0.557143
 0.510153

[:, :, 3] =
 0.584304
 0.539638
 0.638954
 0.741324
 0.720163
 0.75643 
 0.682495
 0.775757
 0.757982
 0.767421
 0.761664
 0.711268
 0.723849
 ⋮       
 0.554444
 0.500086
 0.462988
 0.476145
 0.46434 
 0.50546 
 0.52402 
 

And the metadata :

In [7]:
full_data_dict["jun1417-Fly1-EB"][6][2]

Dict{String,Any} with 3 entries:
  "pulseNumber"   => 30
  "timeToDrug"    => 238000 milliseconds
  "stimIntensity" => 1

#### Processed versions of the results
We're adding one version of the data that is averaged per run (i.e. the repeats are lumped together) :

In [8]:
avg_data_dict = map(full_data_dict) do full  
    (k,full) = full
    meanSignals = [(AxisArray(vec(mean(f[1].data,3)),axes(f[1],Axis{:time})),f[2]) for 
        f in full]
    Pair(k,meanSignals)    
end;

This can be saved locally (and used in other scripts) : (useless for this Binder as we're loading those files from the OSF usually).

In [11]:
## save("data/interpolatedData.jld2",large_interpolated_data_dict)

Then a version of the data where the fluorescence is linearly interpolated on a common time grid. This is used to calculate some statistics (for example correlations between flies) or average experiments together. 2 versions are created, one only contains the times around the stimulation (3 to 5 seconds), the other a large chunk of the recordings (1 to 12 seconds) in 100ms increments :

In [9]:
interpolated_data_dict = map(avg_data_dict) do dct
        (k,val) = dct
        newAvg = [interpolate_run(run,3:0.1:5) for run in val]     ## The two seconds following the stimulation
        Pair(k,newAvg)
end;

large_interpolated_data_dict = map(avg_data_dict) do dct
        (k,val) = dct
        newAvg = [interpolate_run(run,1:0.1:12) for run in val]     ## The two seconds following the stimulation
        Pair(k,newAvg)
end;

Again, this can be saved to be reused by the figures script.

In [12]:
#save("data/interpolatedData.jld2",large_interpolated_data_dict)

#### Calculate statisitics per repeat
For every single experimental repeat, we calculate a battery of statistics (the function ```compileStats``` is defined in ```code/functions/fluoRunUtilities.jl```). 

In [10]:
keyEntries = labbook[:keyEntry];
fullStats = compileStats(full_data_dict,keyEntries)
DataFrames.head(fullStats)

,baseline,integrals,integral_to_peak,integNorm,peakFluo,peakTimes,peakNorm,repeats_correlation,half_decay,nPulses,runIdx,experiment
1,0.666119,0.187951,0.220615,0.323585,0.322164,1.0139,0.483643,0.889083,0.506948,30,1,jun1417-Fly1-EB
2,0.135299,0.334625,0.503745,3.67823,0.649132,1.21667,4.79775,0.889083,0.811116,30,1,jun1417-Fly1-EB
3,0.16728,0.385878,0.501056,2.96198,0.629957,1.11528,3.76588,0.889083,1.0139,30,1,jun1417-Fly1-EB
4,0.0618386,0.5105,0.59548,9.5476,0.838154,1.0139,13.5539,0.889083,1.31806,30,1,jun1417-Fly1-EB
5,0.72332,0.172697,0.125692,0.170267,0.481799,0.506948,0.666094,0.980893,0.912506,20,2,jun1417-Fly1-EB
6,0.216111,0.341538,0.43497,1.99629,0.921512,0.709727,4.26407,0.980893,1.11528,20,2,jun1417-Fly1-EB


#### Statistics per run
We aggregate those statistic per experimental run (each run is constituted by 4 repeats)

In [11]:
stats_per_run = aggregate(fullStats,[:experiment,:runIdx],median);

And reformat and add some metadata to this table :

In [12]:
## Convert the number of pulses back to integer
stats_per_run[:nPulses_median] = convert(Array{Int},stats_per_run[:nPulses_median]);

## Add some metadata columns 
stats_per_run[:cellPair]=""
stats_per_run[:genotype]=""
stats_per_run[:preNeuron]=""
stats_per_run[:postNeuron]=""
stats_per_run[:preDrug]=true
stats_per_run[:Drug]=""
stats_per_run[:timeToDrug]= -Inf
for i in 1:size(stats_per_run,1)
    stats_per_run[i,:cellPair] = labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),:cellToCell]
    stats_per_run[i,:genotype] = labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),:genotypeRegion]
    stats_per_run[i,:preNeuron] = labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),:cellPre]
    stats_per_run[i,:postNeuron] = labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),:cellPost]
    if (!ismissing(labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),:Drug]))
        stats_per_run[i,:preDrug] = (labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),
            :timesToDrug][stats_per_run[i,:runIdx]])>Dates.Second(0)
        stats_per_run[i,:timeToDrug] = -Int64(Dates.value(labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),
            :timesToDrug][stats_per_run[i,:runIdx]]))/60000
        stats_per_run[i,:Drug]=labbook[findfirst(keyEntries.==stats_per_run[i,:experiment]),:Drug]
    end
end

We want to restrict our analysis to cell pairs for which we have a sufficient number of experiments :

In [13]:
## Calculate the number of runs per pair and select pairs with at least 3 runs
ns = by(stats_per_run,:cellPair,df -> DataFrame(n = length(unique(df[:experiment]))))
fullExps = ns[ns[:n].>=3,:cellPair];

stats_per_run = stats_per_run[in.(Array(stats_per_run[:cellPair]),[fullExps]),:];

We also want to categorize the cell pairs in function of their potential synaptic overlaps. For that we use the ```linesAndTypes``` table which has columns for the pre and post-synaptic innervation patterns.

In [14]:
## List the cell types tested
uniqueTypesUsed = unique(vcat(stats_per_run[:preNeuron],stats_per_run[:postNeuron]))
sort!(uniqueTypesUsed,by= v->linesToType[findfirst(linesToType[Symbol("Type Description")].==v),:Supertype] )

## Parse the neuron types to define their pre/post regions
possibleNeuropiles = ["PB","FB","EB","NO","GA","LAL","rub","BU"];
neuronTypes = OrderedDict(uniqueTypesUsed[i] =>
                          Dict("innervates" => filter(x -> contains(uniqueTypesUsed[i],x),
                                                                possibleNeuropiles), 
                               "pre"=> split(linesToType[findfirst(linesToType[:,
                                            Symbol("Type Description")].==uniqueTypesUsed[i]),
                                            Symbol("Pre regions")],","),
                               "post" => split(linesToType[findfirst(linesToType[:,
                                            Symbol("Type Description")].==uniqueTypesUsed[i]),
                                            Symbol("Post regions")],","),
                               "pre_fine"=> split(linesToType[findfirst(linesToType[:,
                                                Symbol("Type Description")].==uniqueTypesUsed[i]),
                                                Symbol("Pre regions fine")],","),
                                "post_fine" => split(linesToType[findfirst(linesToType[:,
                                                Symbol("Type Description")].==uniqueTypesUsed[i]),
                                                Symbol("Post regions fine")],","),
                                "short_name" => linesToType[findfirst(linesToType[:,
                                                    Symbol("Type Description")].==uniqueTypesUsed[i]),
                                                    Symbol("New Type Name")]) for i in 1:length(uniqueTypesUsed))

## Using the annotation, establish if there's a potential overlap for every pair 
stats_per_run[:overlapping]=
[(length(intersect(neuronTypes[ty]["pre_fine"],
        neuronTypes[tyPost]["post_fine"]))>0) for (ty,tyPost) in zip(stats_per_run[:preNeuron],
                                                                     stats_per_run[:postNeuron])]

## Is the same neuron used for recording and stimulation ?
stats_per_run[:self]= (stats_per_run[:preNeuron].==stats_per_run[:postNeuron]);

stats_per_run[:expType] = "Non overlapping"
stats_per_run[stats_per_run[:overlapping],:expType] = "Overlapping"
stats_per_run[stats_per_run[:self],:expType] = "Self stimulation"
stats_per_run[:overlapping] =  stats_per_run[:expType].=="Overlapping"; ## Excluding self activation

We then scale a couple of variables so that inhibition spans the -1 to 0 range while excitation spans 0 to 1 :

In [15]:
stats_per_run[:integNorm_scaled] =  scaleResponse(stats_per_run[:integNorm_median],trimming=true)
stats_per_run[:integral_to_peak_scaled] =  scaleResponse(stats_per_run[:integral_to_peak_median],trimming=true);

```stats_per_run``` now looks like :

In [16]:
DataFrames.head(stats_per_run,3)

,experiment,runIdx,baseline_median,integrals_median,integral_to_peak_median,integNorm_median,peakFluo_median,peakTimes_median,peakNorm_median,repeats_correlation_median,half_decay_median,nPulses_median,cellPair,genotype,preNeuron,postNeuron,preDrug,Drug,timeToDrug,overlapping,self,expType,integNorm_scaled,integral_to_peak_scaled
1,jun1417-Fly1-EB,1,0.15129,0.360251,0.502401,3.3201,0.639544,1.06459,4.28181,0.889083,0.912506,30,PBG2-9.s-EBt.b-NO1.b.Type1-to-PBG2-9.s-EBt.b-NO1.b.Type2,37F06-VK22LexA-SS02232Gal4-Chrimson-in-LexA-EB,PBG2-9.s-EBt.b-NO1.b.Type1,PBG2-9.s-EBt.b-NO1.b.Type2,true,Mecamylamine,-11.6833,true,false,Overlapping,0.0836685,0.0680963
2,jun1417-Fly1-EB,2,0.350915,0.315488,0.461657,1.48336,0.824892,0.760422,2.35963,0.980893,1.16598,20,PBG2-9.s-EBt.b-NO1.b.Type1-to-PBG2-9.s-EBt.b-NO1.b.Type2,37F06-VK22LexA-SS02232Gal4-Chrimson-in-LexA-EB,PBG2-9.s-EBt.b-NO1.b.Type1,PBG2-9.s-EBt.b-NO1.b.Type2,true,Mecamylamine,-10.3167,true,false,Overlapping,0.0373816,0.0625738
3,jun1417-Fly1-EB,3,0.485072,0.0460802,0.168244,0.364218,0.482838,0.506948,1.04294,0.952856,0.861811,10,PBG2-9.s-EBt.b-NO1.b.Type1-to-PBG2-9.s-EBt.b-NO1.b.Type2,37F06-VK22LexA-SS02232Gal4-Chrimson-in-LexA-EB,PBG2-9.s-EBt.b-NO1.b.Type1,PBG2-9.s-EBt.b-NO1.b.Type2,true,Mecamylamine,-8.65,true,false,Overlapping,0.0091785,0.022804


#### Drug experiments tables
We then separate the parts of these tables containing the drug experiments per se (starting 5 minutes before the actual application) :

In [17]:
# Selecting mecamylamine runs    
mecadf = stats_per_run[(stats_per_run[:Drug].=="Mecamylamine") .& (stats_per_run[:timeToDrug].>-5),:]

# Same thing for picrotoxin
picrodf = stats_per_run[(stats_per_run[:Drug].=="Picrotoxin") .& (stats_per_run[:timeToDrug].>-5),:];

## In case we want to save
## save("data/drugTables.jld2","mecadf",mecadf,"picrodf",picrodf,"drugStats",drugStatsDFPerPair)

### Statistics per pairs
We now compile the statistics per cell pair (keeping the different stimulation conditions separate). The ```category_stats``` function used is in ```code/functions/fluoRunUtilities.jl```. We also scale the normalized integral.

In [18]:
stats_per_pair = by(stats_per_run[stats_per_run[:preDrug],:],[:cellPair,:nPulses_median],category_stats)
stats_per_pair[:integNormScaled] = scaleResponse(stats_per_pair[:integNorm]);
DataFrames.head(stats_per_pair,2)

,cellPair,nPulses_median,peakFluo,peakNorm,peakNormMAD,integ,integNorm,integNormMAD,baseline,repeats_corr,peakTime,state_dependence_peak,state_dependence_integral,between_runs_corr,preNeuron,postNeuron,expType,decay_time,n,integNormScaled
1,PBG2-9.s-EBt.b-NO1.b.Type1-to-PBG2-9.s-EBt.b-NO1.b.Type2,30,1.46137,4.88468,4.48307,1.0666,2.75915,4.24766,0.313897,0.96382,0.974527,-0.0471069,-0.0376981,0.971279,PBG2-9.s-EBt.b-NO1.b.Type1,PBG2-9.s-EBt.b-NO1.b.Type2,Overlapping,1.49582,15,0.400995
2,PBG2-9.s-EBt.b-NO1.b.Type1-to-PBG2-9.s-EBt.b-NO1.b.Type2,20,1.89762,8.89445,4.91267,1.23068,6.88077,4.56911,0.217666,0.968653,0.783447,0.185211,0.230812,0.936624,PBG2-9.s-EBt.b-NO1.b.Type1,PBG2-9.s-EBt.b-NO1.b.Type2,Overlapping,1.31887,6,1.0


We then compute the signed distance to the null distribution of responses (as described in the paper) and the significance at 0.01 and 0.005 to the table :

In [19]:
## Which statistics are included in the distance calculation
stats_to_use = [:integNormScaled,
                 :between_runs_corr
               ];

addDistances!(stats_per_pair,stats_to_use,:integNormScaled)

,cellPair,nPulses_median,peakFluo,peakNorm,peakNormMAD,integ,integNorm,integNormMAD,baseline,repeats_corr,peakTime,state_dependence_peak,state_dependence_integral,between_runs_corr,preNeuron,postNeuron,expType,decay_time,n,integNormScaled,signif1,signif5,distance
1,PBG2-9.s-EBt.b-NO1.b.Type1-to-PBG2-9.s-EBt.b-NO1.b.Type2,30,1.46137,4.88468,4.48307,1.0666,2.75915,4.24766,0.313897,0.96382,0.974527,-0.0471069,-0.0376981,0.971279,PBG2-9.s-EBt.b-NO1.b.Type1,PBG2-9.s-EBt.b-NO1.b.Type2,Overlapping,1.49582,15,0.400995,true,true,195.745
2,PBG2-9.s-EBt.b-NO1.b.Type1-to-PBG2-9.s-EBt.b-NO1.b.Type2,20,1.89762,8.89445,4.91267,1.23068,6.88077,4.56911,0.217666,0.968653,0.783447,0.185211,0.230812,0.936624,PBG2-9.s-EBt.b-NO1.b.Type1,PBG2-9.s-EBt.b-NO1.b.Type2,Overlapping,1.31887,6,1.0,true,true,719.42
3,PBG2-9.s-EBt.b-NO1.b.Type1-to-PBG2-9.s-EBt.b-NO1.b.Type2,10,1.30618,5.1324,6.02858,0.648215,0.490713,3.62869,0.185361,0.957765,0.592499,-0.0282337,-0.110286,0.945236,PBG2-9.s-EBt.b-NO1.b.Type1,PBG2-9.s-EBt.b-NO1.b.Type2,Overlapping,1.07634,6,0.0713165,true,true,201.089
4,PBG2-9.s-EBt.b-NO1.b.Type1-to-PBG2-9.s-EBt.b-NO1.b.Type2,1,0.21452,1.01752,1.08219,0.0565002,0.178528,0.261446,0.181559,-0.0404702,0.351361,-0.522664,-0.537753,0.544241,PBG2-9.s-EBt.b-NO1.b.Type1,PBG2-9.s-EBt.b-NO1.b.Type2,Overlapping,0.424945,6,0.0259459,false,false,25.1882
5,PBG2-9.s-EBt.b-NO1.b.Type1-to-PBG2-9.s-EBt.b-NO1.b.Type2,5,0.963049,2.08646,1.83264,0.370436,0.734966,0.462805,0.511499,0.820852,0.501432,-0.00213222,0.182087,0.909641,PBG2-9.s-EBt.b-NO1.b.Type1,PBG2-9.s-EBt.b-NO1.b.Type2,Overlapping,0.848416,6,0.106815,true,true,74.7238
6,PBG1-8.b-EBw.s-DV_GA.b-to-EBORP O-I-GA-Bulb,30,0.450603,1.53855,0.996625,0.323229,0.72302,0.607116,0.365083,0.423462,1.18953,0.697016,0.551504,0.542842,PBG1-8.b-EBw.s-DV_GA.b,EBORP O-I-GA-Bulb,Overlapping,0.489225,22,0.105078,true,true,47.0126
7,PBG1-8.b-EBw.s-DV_GA.b-to-EBORP O-I-GA-Bulb,10,0.519307,0.829359,0.433624,0.136491,0.307981,0.0709725,0.228623,0.130308,0.535155,0.860198,0.642736,0.216901,PBG1-8.b-EBw.s-DV_GA.b,EBORP O-I-GA-Bulb,Overlapping,0.244784,9,0.0447597,true,true,113.05
8,PBG1-8.b-EBw.s-DV_GA.b-to-EBORP O-I-GA-Bulb,5,0.227979,0.447033,0.431115,0.0632756,0.110483,0.0674704,0.325241,0.178601,0.558722,0.868596,0.768607,0.0951973,PBG1-8.b-EBw.s-DV_GA.b,EBORP O-I-GA-Bulb,Overlapping,0.203966,9,0.0160568,false,false,8.86066
9,PBG1-8.b-EBw.s-DV_GA.b-to-EBORP O-I-GA-Bulb,20,0.601913,2.39214,1.06399,0.233377,0.678976,0.821142,0.33227,0.475644,0.802732,0.481351,0.185983,0.424387,PBG1-8.b-EBw.s-DV_GA.b,EBORP O-I-GA-Bulb,Overlapping,0.391654,9,0.0986774,true,true,86.0961
10,PBG1-8.b-EBw.s-DV_GA.b-to-EBORP O-I-GA-Bulb,1,0.0274247,0.0372136,0.504018,0.0098226,0.0255349,0.112198,0.194494,-0.144043,0.30768,0.296509,0.916315,0.0191381,PBG1-8.b-EBw.s-DV_GA.b,EBORP O-I-GA-Bulb,Overlapping,0.196942,9,0.00371105,false,false,1.86737


And add a "global significance" which is the significance at 1% signed by the sign of the response. We also normalize the distance to the maximum response observed at 20 pulses.

In [20]:
stats_per_pair[:globalSignif] =  sign.(stats_per_pair[:distance]).*stats_per_pair[:signif1]
stats_per_pair[stats_per_pair[:globalSignif].==-0.0,:globalSignif]=0
stats_per_pair[:distanceNorm] = stats_per_pair[:distance]./maximum(trim(abs.(stats_per_pair[stats_per_pair[:nPulses_median].<=20,:distance]),
                                                                        prop=0.01));

We then create a data frame restricted to the 20 pulses stimulations, as it's the one we'll be using mainly for the summaries (figures and website).

In [21]:
stats_per_pair_20 = stats_per_pair[stats_per_pair[:nPulses_median].==20,:];

We can then save this data for further use (not run) :

In [35]:
#@save "data/statTables.jld2" stats_per_run stats_per_pair uniqueTypesUsed stats_per_pair_20

## Exports for the website

For exports, we're taking advantage of the JSON package which can translate a Julia dictionary into a json string. For ease of use in the website, we export those json as javascript variables into small .js files (those will be global variables in the website).
- this first function is used to convert the DataFrames into dictionaries :

In [22]:
function get_dataDict_per_key(pk,data_dict)
    dat = data_dict[pk][1:min(6,end)]
    if length(dat)==0
        return(0)
    else
        dat = Dict(x[2]["pulseNumber"] => transformAxisArray(x[1]) for x in dat)
    end
end

get_dataDict_per_key (generic function with 1 method)

It uses the ```transformAxisArray``` methods defined here, that creates an AxisArray into a dictionary (hence a js variable) directly usable by plotly.js which we use for plotting in the website : 

In [23]:
function transformAxisArray{T}(aa::AxisArray{T,1})
    Dict("x"=>axes(aa,1)[:]-axes(aa,1)[findfirst(axes(aa,1)[:].>3.0)],"y"=>aa)
end

transformAxisArray{T}(aa::AxisArray{T,3}) = Dict("x"=>axes(aa,1)[:]-axes(aa,1)[findfirst(axes(aa,1)[:].>3.0)],
                                                 "y"=>reshape(aa,(size(aa,1)*size(aa,2),size(aa,3))))

transformAxisArray{T}(AA::Array{AxisArrays.AxisArray{T,1,Array{T,1},
    Tuple{AxisArrays.Axis{:time,StepRangeLen{Float64}}}},1}) =  Dict("x"=>[axes(aa,1)[:]-axes(aa,1)[findfirst(axes(aa,1)[:].>3.0)] for aa in AA],"y"=>AA)

transformAxisArray (generic function with 3 methods)

We then format and export the data :

In [24]:
## Exporting the full fluorescence and the average
out_data = Dict(pk => get_dataDict_per_key(pk,full_data_dict) for pk in keyEntries)
filter!((x,y) -> y!=0,out_data)
    #writeJS("js/full_data.js","FULL_DATA",out_data)

out_data_avg = Dict(pk => get_dataDict_per_key(pk,avg_data_dict) for pk in keyEntries)
filter!((x,y) -> y!=0,out_data_avg)
    #writeJS("js/avg_data.js","AVG_DATA",out_data_avg)

## Exporting a table mapping the experiments to the cell pairs
pairToExp = Dict(cpair => convert(Array{String},labbook[convert(Array{Bool}
                ,labbook[:cellToCell].==cpair),:keyEntry]) for 
        cpair in unique(labbook[:cellToCell]))
#writeJS("js/pairsToExp.js","PAIRS_TO_EXP",pairToExp)

Dict{String,Array{String,1}} with 76 entries:
  "PBG1-7.s-FBl2.s-LAL.b-c… => String["feb0516-Fly4-LAL", "feb0516-Fly5-LAL", "…
  "PBG1-8.s-EBt.b-DV_GA.b-… => String["jun1217-Fly1-Gall", "jun1217-Fly2-Gall",…
  "PB18.s-GxΔ7Gy.b-PB18.s-… => String["jul0616-Fly1-LAL", "jul0716-Fly5-FB", "j…
  "PBG2-9.s-EBt.b-NO1.b.Ty… => String["dec2016-Fly1-Gall", "dec2116-Fly1-Gall",…
  "PBG1-8.b-EBw.s-DV_GA.b-… => String["jun2716-Fly1-FB", "jun2916-Fly2-FB", "ju…
  "PB18.s-GxΔ7Gy.b-PB18.s-… => String["feb1617-Fly1-EB+NO", "feb1617-Fly2-EB+NO…
  "PBG2-9.b-IB.s.SPS.s-to-… => String["dec0915-Fly1-EB", "dec0915-Fly5-EB", "de…
  "LAL.s-NO3Ai.b-to-LAL.s-… => String["may1416-Fly1-LAL-out", "may1516-Fly7-LAL…
  "LAL.s-GAi.s-NO1i.b-to-P… => String["jul1016-Fly2-FB", "jul1016-Fly3-FB", "ju…
  "PBG1-7.s-FBl2.s-LAL.b-c… => String["jan0416-Fly6-EB", "jan0516-Fly1-EB", "ja…
  "PBG1-8.b-EBw.s-DV_GA.b-… => String["feb0516-Fly2-EB", "feb0516-Fly3-EB", "fe…
  "Wedge-LAL.s-LAL.b.contr… => String["jul1416-Fly2-FB", "jul14

In [25]:
## Exporting a supertype description table
##Limiting to the experiments that have been done
supertypes = unique(linesToType[:Supertype])

superDict = Dict(s => unique(linesToType[(linesToType[:Supertype].==s) .& 
        [td in uniqueTypesUsed for td in linesToType[Symbol("Type Description")]],Symbol("Type Description")]) for 
    s in supertypes)
superDict = filter((k,x) -> !isempty(x),superDict)

#writeJS("js/supertypes.js","SUPERTYPES",superDict)

Dict{String,Array{String,1}} with 10 entries:
  "PB input"       => String["PBG2-9.b-IB.s.SPS.s", "SPS.s-PB.b"]
  "EB input"       => String["EB.w-AMP.d-D_GAsurround"]
  "Other"          => String["EBMRA-FB-LT-LT-GA-GA"]
  "EB columnar"    => String["PBG1-8.b-EBw.s-DV_GA.b", "PBG1-8.s-EBt.b-DV_GA.b"…
  "FB columnar"    => String["PBG2-9.s-FBl3.b-NO2D.b", "PBG2-9.s-FBl1.b-NO3PM.b…
  "FB input"       => String["LAL.s-IMP-FBl3.b"]
  "Ring neuron"    => String["EBIRP I-O-LAL.s", "EBMRP I-O-LAL.s", "EBORP O-I-G…
  "LAL-IN"         => String["SMP.s-LAL.s-LAL.b.contra", "SMP.s-LAL.s-LAL.b.con…
  "PB interneuron" => String["PB18.s-GxΔ7Gy.b-PB18.s-9i1i8c.b"]
  "LAL-NO"         => String["LAL.s-GAi.s-NO1i.b", "Cre.s-LAL.s-NO2.b", "LAL.s-…

Finally the tables :

In [26]:
superSummary = 
Dict(cp => Dict(string(n) => 
stats_per_pair_20[(stats_per_pair_20[:cellPair].==cp),n][1] for 
            n in [:n,:expType,:signif1,:signif5,:distanceNorm]) for
            cp in unique(stats_per_pair_20[:cellPair]))

#drugSummary = 
##Dict(cp => Dict(string(n) => 
#drugStatsDFPerPair[(drugStatsDFPerPair[:cellPair].==cp),n][1] for 
#                n in names(drugStatsDFPerPair)) for
#            cp in unique(drugStatsDFPerPair[:cellPair]))

## For now we're exporting the stats for the drug free runs
perRunDataDict = 
Dict(cp => Dict(nP => Dict(string(n) => 
stats_per_run[(stats_per_run[:cellPair].==cp) .& (stats_per_run[:nPulses_median].==nP) .& (stats_per_run[:preDrug]),
    n] for 
                n in names(stats_per_run)[[collect(3:end)...]]) for
    nP in stats_per_run[stats_per_run[:cellPair].==cp,:nPulses_median]) for 
    cp in unique(stats_per_run[:cellPair]))

## A table of drivers, used by the website
drivers = Dict(td => convert(Array,
        linesToType[linesToType[Symbol("Type Description")].== td,:Line]) for 
                          td in unique(linesToType[Symbol("Type Description")]));
#writeJS("js/drivers.js","DRIVERS",drivers)

#writeJS("js/perRunData.js","PER_RUN_DATA",perRunDataDict)
#writeJS("js/neurontypes.js","NEURON_TYPES",neuronTypes)
#writeJS("js/summaryData.js","SUMMARY_DATA",summaryDataDict)
#writeJS("js/superSummary.js","SUPER_SUMMARY",superSummary)
#writeJS("js/drugSummary.js","DRUG_SUMMARY",drugSummary)